# IFN509 Assignment 2

In [2]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data_file = "Dataset.csv"
df = pd.read_csv(data_file)

## Assessing the shape and dimensions of the dataset

In [4]:
df.head()

,Participant_ID,survey_date,region,country,ip_latitude,ip_longitude,gender,age,height,weight,...,heart_disease,lung_disease,diabetes,hiv_positive,other_chronic,nursing_home,health_worker,risk_infection,risk_infection_level,risk_mortality
0,862217754,02/12/2020,?,US,42.3690,-88.0651,male,30_40,180,98,...,0,0,0,0,0,0,0,100.0,high,0.063
1,2363592,02/07/2020,AF,ZA,-26.2870,28.2427,male,30_40,168,66,...,0,0,1,0,0,0,0,75.0,moderate,0.098
2,2570172,05/07/2020,AF,ZA,-26.2019,28.0273,female,60_70,172,86,...,0,0,0,0,0,0,0,5.0,low,0.582
3,2812446,13/06/2020,AF,ZA,-33.8268,18.4772,male,40_50,172,140,...,1,0,1,0,0,0,0,100.0,high,1.032
4,2867850,29/06/2020,AF,ZA,-26.3120,28.1024,male,20_30,186,78,...,0,0,0,0,0,0,0,5.0,low,0.050


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

# Assessing null values in the dataset

In [5]:
df.isnull().sum()

Participant_ID               0
survey_date                  0
region                      15
country                      3
ip_latitude                  0
ip_longitude                 0
gender                       0
age                          0
height                       0
weight                       0
bmi                          0
blood_type                 100
insurance                 1292
income                    1153
smoking                     36
alcohol                      0
cocaine                   4359
contacts_count              42
public_transport_count    1217
working                     28
worried                   1271
covid19_positive             0
covid19_symptoms             0
covid19_contact              0
asthma                       0
kidney_disease               0
liver_disease                0
compromised_immune           0
heart_disease                0
lung_disease                 0
diabetes                     0
hiv_positive                 0
other_ch